In [1]:
import torch
import matplotlib.pyplot as plt
from torch import optim, distributions, nn
import torch.nn.utils as nn_utils
from tqdm import tqdm
from gpzoo.gp import SVGP, VNNGP
from gpzoo.kernels import NSF_RBF, RBF
from gpzoo.likelihoods import NSF2
from gpzoo.utilities import rescale_spatial_coords, dims_autocorr, regularized_nmf
import squidpy as sq
import numpy as np
import time
import random
import scanpy as sc
import anndata as ad
from anndata import AnnData
from squidpy.gr import spatial_neighbors,spatial_autocorr
from matplotlib.animation import FuncAnimation
from matplotlib import animation
from scipy.stats import pearsonr
from scipy.spatial.distance import cosine
from os import path
import gpzoo.pri_experiment_utilities as pexp

/engelhardt/home/pshrestha/vnngp/GPzoo/gpzoo/utilities.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
!pwd
root_path = '/engelhardt/home/pshrestha/vnngp/'
torch.manual_seed(256)

/engelhardt/home/pshrestha/vnngp/GPzoo/notebooks


In [3]:
print("CUDA available: ", torch.cuda.is_available())
print("Number of GPUs: ", torch.cuda.device_count())
print("Current GPU: ", torch.cuda.current_device())
print("GPU Name: ", torch.cuda.get_device_name(torch.cuda.current_device()))

CUDA available:  True
Number of GPUs:  2
Current GPU:  0
GPU Name:  NVIDIA A30


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [31]:
def rmse(A, B):
    A = np.array(A)
    B = np.array(B)
    diff = A - B
    return np.sqrt(np.mean(np.square(diff)))

In [5]:
L = [5, 10, 15]
N = [3000]
M = [500, 1000, 2000]
K = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lkzz_builds = [1, 2]

In [22]:
X, Y = pexp.load_visium()

In [30]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

save_path = path.join(root_path, "results/visium/nnnsf")
corrs = []
for k in K:
    dicts = {
        'L': 15, 
        'M': 1000, 
        'K': k,
        'sigma': 0.1, 
        'lengthscale': 0.07, 
        'jtr': 1e-2,
        'batch_size': 128,
        'lr': 1e-3,
        'rs': 256,
        'lkzz_build': 1}
    factors, loadings = pexp.preloaded_nmf_factors("/engelhardt/home/pshrestha/vnngp/results/visium/nmf/nmf_factors_iter=1000_rs=256_L=15.pt")
    NNNSF = pexp.build_model(X, Y, loadings=loadings, factors=factors, model_type='VNNGP', kwargs=dicts)
    pexp.model_grads(NNNSF)
    NNNSF.prior.jitter = dicts['jtr']
    file_path = path.join(save_path, f"VNNGP_K={dicts['K']}_lkzz={dicts['lkzz_build']}_M={dicts['M']}_L={dicts['L']}_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth")
    NNNSF.load_state_dict(torch.load(file_path))
    X = torch.tensor(X).type(torch.float)
    X = X.clone().detach().type(torch.float)
    NNNSF.cpu()
    qF, _, _ = NNNSF.prior(X)
    
    means = torch.exp(qF.mean).detach().numpy() # means = factors
    W = (NNNSF.W).cpu()
    W_transformed = nn.functional.softplus(W.T)

    W_transformd = W_transformed = W_transformed.detach().numpy()
    y_nmf = (np.exp(means.T)).dot(W_transformed)

    corr_xy, p_value = pearsonr(Y.flatten(), (y_nmf.T).flatten())
    corrs.append(corr_xy)
    print(f"With K={str(k)} pearson correlation is: {str(corr_xy)}")

With K=1 pearson correlation is: 0.8489027029845029
With K=2 pearson correlation is: 0.8497458723259091
With K=3 pearson correlation is: 0.849085474064595
With K=4 pearson correlation is: 0.848527065652418
With K=5 pearson correlation is: 0.8478300463930877
With K=6 pearson correlation is: 0.8475892494629158
With K=7 pearson correlation is: 0.8474132423789089
With K=8 pearson correlation is: 0.847377006854974
With K=9 pearson correlation is: 0.847420706632707
With K=10 pearson correlation is: 0.8475344597776198


In [34]:
 import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

save_path = path.join(root_path, "results/visium/nnnsf")
corrs = []
for k in K:
    dicts = {
        'L': 15, 
        'M': 1000, 
        'K': k,
        'sigma': 0.1, 
        'lengthscale': 0.07, 
        'jtr': 1e-2,
        'batch_size': 128,
        'lr': 1e-3,
        'rs': 256,
        'lkzz_build': 1}
    factors, loadings = pexp.preloaded_nmf_factors("/engelhardt/home/pshrestha/vnngp/results/visium/nmf/nmf_factors_iter=1000_rs=256_L=15.pt")
    NNNSF = pexp.build_model(X, Y, loadings=loadings, factors=factors, model_type='VNNGP', kwargs=dicts)
    pexp.model_grads(NNNSF)
    NNNSF.prior.jitter = dicts['jtr']
    file_path = path.join(save_path, f"VNNGP_K={dicts['K']}_lkzz={dicts['lkzz_build']}_M={dicts['M']}_L={dicts['L']}_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth")
    NNNSF.load_state_dict(torch.load(file_path))
    X = torch.tensor(X).type(torch.float)
    X = X.clone().detach().type(torch.float)
    NNNSF.cpu()
    qF, _, _ = NNNSF.prior(X)
    
    means = torch.exp(qF.mean).detach().numpy() # means = factors
    W = (NNNSF.W).cpu()
    W_transformed = nn.functional.softplus(W.T)

    W_transformd = W_transformed = W_transformed.detach().numpy()
    y_nmf = (np.exp(means.T)).dot(W_transformed)

    #corr_xy, p_value = pearsonr(Y.flatten(), (y_nmf.T).flatten())
    r = rmse(Y, y_nmf.T)
    corrs.append(r)
    print(f"With K={str(k)} rmse is: {str(r)}")

With K=1 rmse is: 434.0536283218856
With K=2 rmse is: 375.6886338309844
With K=3 rmse is: 351.01416044643815
With K=4 rmse is: 340.0835252299416
With K=5 rmse is: 335.3084163293762
With K=6 rmse is: 333.28849158840177
With K=7 rmse is: 332.6239167670247
With K=8 rmse is: 332.6454927047871
With K=9 rmse is: 333.07358223744274
With K=10 rmse is: 333.6842771184943


In [36]:
 import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

save_path = path.join(root_path, "results/visium/nnnsf")
corrs = []
for k in K:
    dicts = {
        'L': 15, 
        'M': 1000, 
        'K': k,
        'sigma': 0.1, 
        'lengthscale': 0.07, 
        'jtr': 1e-2,
        'batch_size': 128,
        'lr': 1e-3,
        'rs': 256,
        'lkzz_build': 1}
    factors, loadings = pexp.preloaded_nmf_factors("/engelhardt/home/pshrestha/vnngp/results/visium/nmf/nmf_factors_iter=1000_rs=256_L=15.pt")
    NNNSF = pexp.build_model(X, Y, loadings=loadings, factors=factors, model_type='VNNGP', kwargs=dicts)
    pexp.model_grads(NNNSF)
    NNNSF.prior.jitter = dicts['jtr']
    file_path = path.join(save_path, f"VNNGP_K={dicts['K']}_lkzz={dicts['lkzz_build']}_M={dicts['M']}_L={dicts['L']}_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth")
    NNNSF.load_state_dict(torch.load(file_path))
    X = torch.tensor(X).type(torch.float)
    X = X.clone().detach().type(torch.float)
    NNNSF.cpu()
    qF, _, _ = NNNSF.prior(X)
    
    means = torch.exp(qF.mean).detach().numpy() # means = factors
    W = (NNNSF.W).cpu()
    W_transformed = nn.functional.softplus(W.T)

    W_transformd = W_transformed = W_transformed.detach().numpy()
    y_nmf = (np.exp(means.T)).dot(W_transformed)

    #corr_xy, p_value = pearsonr(Y.flatten(), (y_nmf.T).flatten())
    cosine_similarity = 1 - cosine(Y.flatten(), y_nmf.T.flatten())
    corrs.append(cosine_similarity)
    print(f"With K={str(k)} cosine similarity is: {str(cosine_similarity)}")

With K=1 cosine similarity is: 0.8493583854953297
With K=2 cosine similarity is: 0.8502559339341007
With K=3 cosine similarity is: 0.8496515995897862
With K=4 cosine similarity is: 0.8491311423491493
With K=5 cosine similarity is: 0.8484565065727885
With K=6 cosine similarity is: 0.8482246310688355
With K=7 cosine similarity is: 0.8480523398466375
With K=8 cosine similarity is: 0.8480167617980309
With K=9 cosine similarity is: 0.8480586504460711
With K=10 cosine similarity is: 0.8481693850914389


In [39]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

save_path = path.join(root_path, "results/visium/nnnsf")
corrs = []
for k in K:
    dicts = {
        'L': 5, 
        'M': 1000, 
        'K': k,
        'sigma': 0.1, 
        'lengthscale': 0.07, 
        'jtr': 1e-2,
        'batch_size': 128,
        'lr': 1e-3,
        'rs': 256,
        'lkzz_build': 1}
    factors, loadings = pexp.preloaded_nmf_factors(f"/engelhardt/home/pshrestha/vnngp/results/visium/nmf/nmf_factors_iter=1000_rs=256_L={dicts['L']}.pt")
    NNNSF = pexp.build_model(X, Y, loadings=loadings, factors=factors, model_type='VNNGP', kwargs=dicts)
    pexp.model_grads(NNNSF)
    NNNSF.prior.jitter = dicts['jtr']
    file_path = path.join(save_path, f"VNNGP_K={dicts['K']}_lkzz={dicts['lkzz_build']}_M={dicts['M']}_L={dicts['L']}_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth")
    NNNSF.load_state_dict(torch.load(file_path))
    X = torch.tensor(X).type(torch.float)
    X = X.clone().detach().type(torch.float)
    NNNSF.cpu()
    qF, _, _ = NNNSF.prior(X)
    
    means = torch.exp(qF.mean).detach().numpy() # means = factors
    W = (NNNSF.W).cpu()
    W_transformed = nn.functional.softplus(W.T)

    W_transformd = W_transformed = W_transformed.detach().numpy()
    y_nmf = (np.exp(means.T)).dot(W_transformed)

    corr_xy, p_value = pearsonr(Y.flatten(), (y_nmf.T).flatten())
    corrs.append(corr_xy)
    print(f"With K={str(k)} pearson correlation is: {str(corr_xy)}")

With K=1 pearson correlation is: 0.8483255814189644
With K=2 pearson correlation is: 0.8519319964679883
With K=3 pearson correlation is: 0.8533091136390737
With K=4 pearson correlation is: 0.8539104860553656
With K=5 pearson correlation is: 0.8535384412723173
With K=6 pearson correlation is: 0.8534872889634493
With K=7 pearson correlation is: 0.8533453091557475
With K=8 pearson correlation is: 0.8533145611491434
With K=9 pearson correlation is: 0.8533507098935394
With K=10 pearson correlation is: 0.8534386636480769


In [40]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

save_path = path.join(root_path, "results/visium/nnnsf")
corrs = []
for k in K:
    dicts = {
        'L': 10, 
        'M': 1000, 
        'K': k,
        'sigma': 0.1, 
        'lengthscale': 0.07, 
        'jtr': 1e-2,
        'batch_size': 128,
        'lr': 1e-3,
        'rs': 256,
        'lkzz_build': 1}
    factors, loadings = pexp.preloaded_nmf_factors(f"/engelhardt/home/pshrestha/vnngp/results/visium/nmf/nmf_factors_iter=1000_rs=256_L={dicts['L']}.pt")
    NNNSF = pexp.build_model(X, Y, loadings=loadings, factors=factors, model_type='VNNGP', kwargs=dicts)
    pexp.model_grads(NNNSF)
    NNNSF.prior.jitter = dicts['jtr']
    file_path = path.join(save_path, f"VNNGP_K={dicts['K']}_lkzz={dicts['lkzz_build']}_M={dicts['M']}_L={dicts['L']}_lr=0.001_jtr=0.01_iter=6000_bs=128_NMFinit_state_dict.pth")
    NNNSF.load_state_dict(torch.load(file_path))
    X = torch.tensor(X).type(torch.float)
    X = X.clone().detach().type(torch.float)
    NNNSF.cpu()
    qF, _, _ = NNNSF.prior(X)
    
    means = torch.exp(qF.mean).detach().numpy() # means = factors
    W = (NNNSF.W).cpu()
    W_transformed = nn.functional.softplus(W.T)

    W_transformd = W_transformed = W_transformed.detach().numpy()
    y_nmf = (np.exp(means.T)).dot(W_transformed)

    corr_xy, p_value = pearsonr(Y.flatten(), (y_nmf.T).flatten())
    corrs.append(corr_xy)
    print(f"With K={str(k)} pearson correlation is: {str(corr_xy)}")

With K=1 pearson correlation is: 0.8472383262080498
With K=2 pearson correlation is: 0.8491324046884164
With K=3 pearson correlation is: 0.8491643544815329
With K=4 pearson correlation is: 0.8489790019042032
With K=5 pearson correlation is: 0.8483756357497512
With K=6 pearson correlation is: 0.848199634396712
With K=7 pearson correlation is: 0.8480317557419632
With K=8 pearson correlation is: 0.847992793165879
With K=9 pearson correlation is: 0.8480339796425365
With K=10 pearson correlation is: 0.8481434682187617
